In [1]:
# import libraries

import pandas as pd
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
# connect to Redshift

load_dotenv()
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [4]:
# query online_transactions and stock_description tables from bootcamp schema

query = """
select *
from bootcamp.online_transactions
"""

online_transactions=pd.read_sql(query, connect)
online_transactions

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
0,536366,22632,6,2010-12-01 08:28:00,1.85,u1785,United Kingdom
1,536367,22749,8,2010-12-01 08:34:00,3.75,u13047,United Kingdom
2,536367,22622,2,2010-12-01 08:34:00,9.95,u13047,United Kingdom
3,536370,22659,24,2010-12-01 08:45:00,1.95,u12583,France
4,536370,22492,36,2010-12-01 08:45:00,0.65,u12583,France
...,...,...,...,...,...,...,...
541905,581580,22906,1,2011-12-09 12:20:00,1.65,u12748,United Kingdom
541906,581580,23338,1,2011-12-09 12:20:00,2.08,u12748,United Kingdom
541907,581580,22721,1,2011-12-09 12:20:00,1.95,u12748,United Kingdom
541908,581585,23356,3,2011-12-09 12:31:00,5.95,u15804,United Kingdom


In [5]:
query="""
select *
from bootcamp.stock_description
"""

stock_description=pd.read_sql(query, connect)
stock_description

,stock_code,description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE
...,...,...
3947,DCGSSGIRL,GIRLS PARTY BAG
3948,DOT,DOTCOM POSTAGE
3949,PADS,PADS TO MATCH ALL CUSHIONS
3950,POST,POSTAGE


#### Exploring Data

In [6]:
online_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       541910 non-null  object 
 1   stock_code    541910 non-null  object 
 2   quantity      541910 non-null  int64  
 3   invoice_date  541910 non-null  object 
 4   price         541910 non-null  float64
 5   customer_id   541910 non-null  object 
 6   country       541910 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 28.9+ MB


In [7]:
online_transactions.describe()

,quantity,price
count,541910.000000,541910.000000
mean,9.552234,4.611138
std,218.080957,96.759765
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [8]:
online_transactions.isna().sum()

invoice         0
stock_code      0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
dtype: int64

In [9]:
online_transactions.duplicated().sum()

5270

In [10]:
online_transactions[online_transactions.price < 0]

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
212125,A563187,B,1,2011-08-12 14:52:00,-11062.06,,United Kingdom
213916,A563186,B,1,2011-08-12 14:51:00,-11062.06,,United Kingdom


In [11]:
# 1.from the table above we can see that some customer_id's have null values! We're gonna clean it after



In [11]:
online_transactions[online_transactions.quantity < 0].sort_values(by="quantity", ascending=True)

,invoice,stock_code,quantity,invoice_date,price,customer_id,country
541813,C581484,23843,-80995,2011-12-09 09:27:00,2.08,u16446,United Kingdom
173986,C541433,23166,-74215,2011-01-18 10:17:00,1.04,u12346,United Kingdom
117855,556690,23005,-9600,2011-06-14 10:37:00,0.00,,United Kingdom
79318,556691,23005,-9600,2011-06-14 10:37:00,0.00,,United Kingdom
2600,C536757,84347,-9360,2010-12-02 14:23:00,0.03,u15838,United Kingdom
...,...,...,...,...,...,...,...
83903,C540270,20725,-1,2011-01-06 11:40:00,1.45,u1468,United Kingdom
83904,C540271,M,-1,2011-01-06 11:51:00,1126.00,u12503,Spain
372712,C556268,22180,-1,2011-06-09 19:34:00,9.95,u13317,United Kingdom
372987,C578007,23234,-1,2011-11-22 12:40:00,2.89,u16474,United Kingdom


In [12]:
# 2.We need to remove this quantity values under 0

In [13]:
stock_description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3952 entries, 0 to 3951
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   stock_code   3952 non-null   object
 1   description  3952 non-null   object
dtypes: object(2)
memory usage: 61.9+ KB


In [14]:
stock_description.describe()

,stock_code,description
count,3952,3952
unique,3905,3785
top,22600,?
freq,2,47


In [19]:
# 3.Here is '?' in description, need to fix it

In [20]:
stock_description.isna().sum()

stock_code     0
description    0
dtype: int64

In [21]:
stock_description.duplicated().sum()

0

In [22]:
# find missing stock codes in stock description

query="""
select distinct ot.stock_code
from bootcamp.online_transactions ot
left join bootcamp.stock_description sd
on ot.stock_code = sd.stock_code
where sd.stock_code is null and
customer_id <>''
"""
pd.read_sql(query, connect)

,stock_code
0,21705
1,84247K
2,D
3,21704
4,46000S
5,22889
6,16151A
7,CRUK
8,23702
9,22686


In [23]:
# 4.get rid of B, M, BANK CHARGES AND CRUK

#### Cleaning Data

In [27]:
# 4 Things to do: replace '?' with 'Unknown', remove values under 0, null values for customer_id and missing stock codes

In [28]:
query = """
SELECT ot.*,
CASE WHEN sd.description is null or sd.description = '?' then 'Unknown' 
ELSE sd.description END AS description
FROM bootcamp.online_transactions ot
LEFT JOIN bootcamp.stock_description sd 
ON ot.stock_code = sd.stock_code
WHERE ot.customer_id <>'' AND
ot.stock_code NOT IN ('M','D','BANK CHARGES','CRUK') AND
ot.quantity > 0
"""
ot_w_sd=pd.read_sql(query,connect)
ot_w_sd.head()

,invoice,stock_code,quantity,invoice_date,price,customer_id,country,description
0,536365,84029E,6,2010-12-01 08:26:00,3.39,u1785,United Kingdom,RED WOOLLY HOTTIE WHITE HEART.
1,536370,22900,24,2010-12-01 08:45:00,2.95,u12583,France,SET 2 TEA TOWELS I LOVE LONDON
2,536373,21730,6,2010-12-01 09:02:00,4.25,u1785,United Kingdom,GLASS STAR FROSTED T-LIGHT HOLDER
3,536375,84406B,8,2010-12-01 09:32:00,2.75,u1785,United Kingdom,CREAM CUPID HEARTS COAT HANGER
4,536378,21212,120,2010-12-01 09:37:00,0.42,u14688,United Kingdom,PACK OF 72 RETROSPOT CAKE CASES


In [35]:
ot_w_sd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405135 entries, 0 to 405134
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       405135 non-null  object 
 1   stock_code    405135 non-null  object 
 2   quantity      405135 non-null  int64  
 3   invoice_date  405135 non-null  object 
 4   price         405135 non-null  float64
 5   customer_id   405135 non-null  object 
 6   country       405135 non-null  object 
 7   description   405135 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 24.7+ MB


In [36]:
ot_w_sd.duplicated().sum()

5268

In [39]:
# drop duplicates
ot_w_sd_cleaned=ot_w_sd.drop_duplicates(keep="first")

In [40]:
ot_w_sd_cleaned.duplicated().sum()

0

In [41]:
ot_w_sd_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399867 entries, 0 to 405134
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice       399867 non-null  object 
 1   stock_code    399867 non-null  object 
 2   quantity      399867 non-null  int64  
 3   invoice_date  399867 non-null  object 
 4   price         399867 non-null  float64
 5   customer_id   399867 non-null  object 
 6   country       399867 non-null  object 
 7   description   399867 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 27.5+ MB


In [42]:
# invoice_date is object, convert it to datetime
ot_w_sd_cleaned["invoice_date"]=pd.to_datetime(ot_w_sd_cleaned['invoice_date'])

In [43]:
ot_w_sd_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399867 entries, 0 to 405134
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   invoice       399867 non-null  object        
 1   stock_code    399867 non-null  object        
 2   quantity      399867 non-null  int64         
 3   invoice_date  399867 non-null  datetime64[ns]
 4   price         399867 non-null  float64       
 5   customer_id   399867 non-null  object        
 6   country       399867 non-null  object        
 7   description   399867 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 27.5+ MB
